In [ ]:
#correcion de prediccion palabra ambigua
#fusionar los dos archivos

#  Importar librerías
import pandas as pd

# Cargar archivos
df_palabras = pd.read_csv("/content/top_300_ambiguas_completo.csv")
df_frases = pd.read_csv("/content/frases_traducidas_500.csv")

# Unir por palabra ambigua (palabra/frase)
df_merged = pd.merge(df_frases, df_palabras[["Palabra", "Significados_es"]],
                     left_on="palabra", right_on="Palabra", how="left")

# Eliminar la columna duplicada 'Palabra' del diccionario
df_merged.drop(columns=["Palabra"], inplace=True)

# Eliminar filas que NO tengan traducción o lista de significados
df_merged = df_merged.dropna(subset=["traduccion_es", "Significados_es"])

# Eliminar filas con lista de significados vacía
df_merged = df_merged[df_merged["Significados_es"].str.strip() != ""]

# Mostrar cuántas frases útiles quedaron
print(f"Frases útiles después de limpiar: {df_merged.shape[0]}")
df_merged.head()


print(f"Frases útiles para etiquetar: {df_merged.shape[0]}")
df_merged.head()

# Verificar fusión
print(df_merged.shape)
print(df_merged.columns)
df_merged.head()

In [ ]:
#guardamos frases fusionadas
df_merged.to_csv("frases_fusionadas.csv", index=False)

In [ ]:
#etiquetado semantico
# Instalar spaCy y el modelo en español
!pip install -U spacy
!python -m spacy download es_core_news_md

In [ ]:
import spacy
from tqdm import tqdm

# Cargar el modelo de español
nlp = spacy.load("es_core_news_md")
tqdm.pandas()

In [ ]:
# Función para seleccionar el significado más similar a la traducción
def predecir_significado(fila):
    if pd.isna(fila["Significados_es"]) or pd.isna(fila["traduccion_es"]):
        return "NO_DISPONIBLE"

    # Separar significados como lista
    significados = [s.strip() for s in fila["Significados_es"].split(";") if s.strip()]
    if not significados:
        return "SIN_SIGNIFICADOS"

    # Comparar con traducción
    doc_trad = nlp(fila["traduccion_es"])
    max_score = 0
    mejor_significado = "NO_DETECTADO"

    for sig in significados:
        doc_sig = nlp(sig)
        score = doc_trad.similarity(doc_sig)
        if score > max_score:
            max_score = score
            mejor_significado = sig

    return mejor_significado

In [ ]:
#aplicar la funcion al dataset

In [ ]:
df_merged["significado_predicho"] = df_merged.progress_apply(predecir_significado, axis=1)

In [ ]:
df_merged.to_csv("frases_etiquetadas_corregidas.csv", index=False)
print(" Archivo guardado como frases_etiquetadas_corregidas.csv")

In [ ]:
#entrenamos el modelo con el nuevo data set
#objetivo es entrenar un modelo que, dada una frase y una palabra ambigua, prediga el significado correcto en contexto.
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report


In [ ]:
# Cargar el archivo limpio y etiquetado
df = pd.read_csv("frases_etiquetadas_corregidas.csv")

# Usamos solo las columnas necesarias
df = df[["frase", "palabra", "significado_predicho"]].dropna()

In [ ]:
#Combinamos la frase completa con la palabra ambigua marcada explícitamente con el texto [PALABRA]
#para que el modelo sepa que palabra estamos desambiguando
df["input_text"] = df["frase"] + " [PALABRA] " + df["palabra"]

In [ ]:
# Convertimos texto a vectores numéricos
vectorizer = TfidfVectorizer(max_features=3000)
X = vectorizer.fit_transform(df["input_text"])

# Variable objetivo
y = df["significado_predicho"]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
#entrenamiento
clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, y_train)

In [ ]:
#evaluamos el rendimiento del modelo
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

In [ ]:
#predice mal. asi que cargamos en dataset y vemos las 20 clase mas frecuentes

In [ ]:
import pandas as pd

# Cargar archivo
df = pd.read_csv("/content/frases_etiquetadas_corregidas.csv")

# Asegurar que las columnas estén bien y eliminar nulos
df = df[["frase", "palabra", "significado_predicho"]].dropna()

# Ver las 20 clases más frecuentes
top_clases = df["significado_predicho"].value_counts().nlargest(20)
print(top_clases)

In [ ]:
# Filtrar el dataset solo con las clases más comunes
df_filtrado = df[df["significado_predicho"].isin(top_clases.index)].copy()

# Crear texto de entrada combinando frase + palabra ambigua
df_filtrado["input_text"] = df_filtrado["frase"] + " [PALABRA] " + df_filtrado["palabra"]

# Verificar
df_filtrado.head()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

# Vectorizar texto con TF-IDF
vectorizer = TfidfVectorizer(max_features=3000)
X = vectorizer.fit_transform(df_filtrado["input_text"])
y = df_filtrado["significado_predicho"]

# División entrenamiento/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenar modelo
clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

# Predicciones
y_pred = clf.predict(X_test)

# Reporte de métricas
print(classification_report(y_test, y_pred))

# (opcional) Matriz de confusión
import seaborn as sns
import matplotlib.pyplot as plt

cm = confusion_matrix(y_test, y_pred, labels=clf.classes_)
plt.figure(figsize=(12, 8))
sns.heatmap(cm, annot=True, fmt="d", xticklabels=clf.classes_, yticklabels=clf.classes_, cmap="Blues")
plt.title("Matriz de confusión")
plt.ylabel("Etiqueta verdadera")
plt.xlabel("Predicción")
plt.show()

In [ ]:
#tampoco predeci bien asi que vamos a utilizar un modelo mas complejo
!pip install -U sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Usar modelo multilingüe optimizado
model = SentenceTransformer('distiluse-base-multilingual-cased-v1')

# Codificar frases
X = model.encode(df_filtrado["input_text"].tolist())
y = df_filtrado["significado_predicho"].tolist()

# Dividir en entrenamiento y test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenar modelo
clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, y_train)

# Evaluar
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

In [ ]:
#intentar mejoras
# Filtrar clases con al menos 8 ejemplos
top_classes = df["significado_predicho"].value_counts()
clases_validas = top_classes[top_classes >= 8].index
df_filtrado = df[df["significado_predicho"].isin(clases_validas)].copy()

# Crear input_text de nuevo (si no lo tenías hecho)
df_filtrado["input_text"] = df_filtrado["frase"] + " [PALABRA] " + df_filtrado["palabra"]

# Ver clases seleccionadas
print(df_filtrado["significado_predicho"].value_counts())

In [ ]:
# Crear columna agrupada
def agrupar_significado(texto):
    if "terminar" in texto:
        return "acción_final"
    elif "muy bien" in texto or "bien" in texto:
        return "aprobación"
    elif "de ninguna manera" in texto or "completamente (no)" in texto:
        return "negación"
    elif "con" in texto or "a lo largo de" in texto:
        return "relación"
    elif "el más" in texto or "tan pronto como" in texto:
        return "comparación"
    elif "querer" in texto or "necesitar" in texto:
        return "intención"
    elif "siempre" in texto or "todo" in texto:
        return "generalidad"
    elif "verdadero" in texto:
        return "veracidad"
    else:
        return "otros"

# Aplicar agrupación
df["significado_agrupado"] = df["significado_predicho"].apply(agrupar_significado)

# Filtrar para entrenar solo con agrupados
df_filtrado = df[df["significado_agrupado"] != "otros"].copy()
df_filtrado["input_text"] = df_filtrado["frase"] + " [PALABRA] " + df_filtrado["palabra"]

# Ver conteo
print(df_filtrado["significado_agrupado"].value_counts())

In [ ]:
y = df_filtrado["significado_predicho"]  # para opción A
y = df_filtrado["significado_agrupado"]  # para opción B

In [ ]:
# Guardar el dataset final
df_filtrado.to_csv("dataset_entrenamiento_final.csv", index=False)

In [ ]:
from google.colab import files
files.download("dataset_entrenamiento_final.csv")

In [ ]:
!pip install -U sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Usar modelo multilingüe optimizado
model = SentenceTransformer('distiluse-base-multilingual-cased-v1')

# Codificar frases
X = model.encode(df_filtrado["input_text"].tolist())
y = df_filtrado["significado_predicho"].tolist()

# Dividir en entrenamiento y test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenar modelo
clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, y_train)

# Evaluar
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

In [ ]:
#Ha mejorado un poco.